<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2021/blob/main/600/pytorch_advanced-revised/4_pose_estimation/GC4_6_OpenPose_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.6 学習と検証の実施

- 本ファイルでは、OpenPoseの学習と検証の実施を行います。AWSのGPUマシンで計算します。
- p2.xlargeで45分ほどかかります。


# 学習目標

1.	OpenPoseの学習を実装できるようになる

---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

Change to the JST notation.


In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd 202107_Tool-A/Work600/
!ls -l

Start mounting your Google Drive.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
Move to the working directory.
/content/drive/My Drive/202107_Tool-A/Work600
total 4
drwx------ 13 root root 4096 Jul 31 09:41 pytorch_advanced


---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

OK. Alreadly git cloned. You can go.


In [ ]:
!ls

pytorch_advanced


In [ ]:
%cd "pytorch_advanced"

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced


In [ ]:
!ls

1_image_classification	 7_nlp_sentiment_transformer
2_objectdetection	 8_nlp_sentiment_bert
3_semantic_segmentation  9_video_classification_eco
4_pose_estimation	 etc
5_gan_generation	 LICENSE
6_gan_anomaly_detection  README.md


In [ ]:
%cd "4_pose_estimation"

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation


In [ ]:
!ls

4-2_DataLoader.ipynb	     4-7_OpenPose_inference.ipynb	    utils
4-3-4_NetworkModel.ipynb     data				    weights
4-5_TensorBoardX.ipynb	     make_folders_and_data_downloads.ipynb
4-6_OpenPose_training.ipynb  tbX


---
# Utils, Data, and Weights

Working directories are:

* /root/utils  
* /root/data  
* /root/weights  

val2014.zip and mask.tar.gz are expanded there.

In [ ]:
!ls -l '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/'

total 2628
-rw------- 1 root root 1273983 Jul 31 09:42 4-2_DataLoader.ipynb
-rw------- 1 root root  150239 Jul 31 09:42 4-3-4_NetworkModel.ipynb
-rw------- 1 root root   15393 Jul 31 09:42 4-5_TensorBoardX.ipynb
-rw------- 1 root root   14793 Jul 31 09:42 4-6_OpenPose_training.ipynb
-rw------- 1 root root 1213399 Jul 31 09:42 4-7_OpenPose_inference.ipynb
drwx------ 2 root root    4096 Jul 31 09:42 data
-rw------- 1 root root    5310 Jul 31 09:42 make_folders_and_data_downloads.ipynb
drwx------ 2 root root    4096 Aug  1 14:51 tbX
drwx------ 3 root root    4096 Jul 31 09:42 utils
drwx------ 2 root root    4096 Aug  1 12:49 weights


In [ ]:
# This will take 8 minutes.

%cd ~

!cp -r '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/utils/'   .
!cp -r '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/data/'    .
!cp -r '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/weights/' .

%cd data
!unzip -q -o  val2014.zip
!tar xfz mask.tar.gz

%cd ~

/root


In [ ]:
!ls -l utils data weights

---
# 事前準備

- これまでの章で実装したクラスと関数をフォルダ「utils」内に用意しています


In [ ]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)


# DataLoader作成

In [ ]:
# about 30 sec
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

# MS COCOのファイルパスリスト作成
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(
    rootpath="./data/")

# Dataset作成
# 本書ではデータ量の問題から、trainをval_listで作成している点に注意
train_dataset = COCOkeypointsDataset(
    val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())

# 今回は簡易な学習とし検証データは作成しない
# val_dataset = CocokeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())

# DataLoader作成
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

# val_dataloader = data.DataLoader(
#    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
# dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}
dataloaders_dict = {"train": train_dataloader, "val": None}


# ネットワークモデル作成

In [ ]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()


# 損失関数を定義

In [ ]:
# 損失関数の設定
class OpenPoseLoss(nn.Module):
    """OpenPoseの損失関数のクラスです。"""

    def __init__(self):
        super(OpenPoseLoss, self).__init__()

    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        """
        損失関数の計算。

        Parameters
        ----------
        saved_for_loss : OpenPoseNetの出力(リスト)

        heatmap_target : [num_batch, 19, 46, 46]
            正解の部位のアノテーション情報

        heatmap_mask : [num_batch, 19, 46, 46]
            heatmap画像のmask

        paf_target : [num_batch, 38, 46, 46]
            正解のPAFのアノテーション情報

        paf_mask : [num_batch, 38, 46, 46]
            PAF画像のmask

        Returns
        -------
        loss : テンソル
            損失の値
        """

        total_loss = 0
        # ステージごとに計算します
        for j in range(6):

            # PAFsとheatmapsにおいて、マスクされている部分（paf_mask=0など）は無視させる
            # PAFs
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask

            # heatmaps
            pred2 = saved_for_loss[2 * j + 1] * heat_mask
            gt2 = heatmap_target.float()*heat_mask

            total_loss += F.mse_loss(pred1, gt1, reduction='mean') + \
                F.mse_loss(pred2, gt2, reduction='mean')

        return total_loss


criterion = OpenPoseLoss()


# 最適化手法を設定

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=1e-2,
                      momentum=0.9,
                      weight_decay=0.0001)


# 学習を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # 画像の枚数
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # イテレーションカウンタをセット
    iteration = 1

    # epochのループ
    for epoch in range(num_epochs):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                optimizer.zero_grad()
                print('（train）')

            # 今回は検証はスキップ
            else:
                continue
                # net.eval()   # モデルを検証モードに
                # print('-------------')
                # print('（val）')

            # データローダーからminibatchずつ取り出すループ
            for imges, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                # ミニバッチがサイズが1だと、バッチノーマライゼーションでエラーになるのでさける
                if imges.size()[0] == 1:
                    continue

                # GPUが使えるならGPUにデータを送る
                imges = imges.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # (out6_1, out6_2)は使わないので _ で代替
                    _, saved_for_loss = net(imges)

                    loss = criterion(saved_for_loss, heatmap_target,
                                     heat_mask, paf_target, paf_mask)
                    del saved_for_loss
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    # else:
                        #epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, 0))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/openpose_net_' +
               str(epoch+1) + '.pth')


In [ ]:
# 学習・検証を実行する
# 学習の雰囲気を感じるためだけなので、学習セットも小さくecoch数も小さい
# It will take 50 minutes or so (25 min per epoch).

# Just to surpress UserWarning
import warnings
warnings.simplefilter('ignore')

num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2021/08/02. 